--- 
0. Setting up the environment
--- 

In [1]:
# import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import cloudscraper
import json
import os

--- 
01. Getting the links for the pages
--- 

In [2]:
def read_cloud(link):
    scraper = cloudscraper.create_scraper()
    response = scraper.get(link)
    return(BeautifulSoup(response.text, 'html'))

In [3]:
base_url = "https://www.payscale.com"
main_url = "https://www.payscale.com/research/US/Job/Information-Technology"

In [4]:
# the main url has many pages, get all of them
abc = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W']

In [5]:
def get_links(url):
    soups = [read_cloud(url) for url in [f'{main_url}/{letter}' for letter in abc]] # this gives a list of soups
    list_of_list_urls = [x.find_all(class_ ='subcats__links__item') for x in soups] # this gives a list of lists
    main_urls = [x['href'] for y in list_of_list_urls for x in y] # this flattens the list of lists
    main_urls = [f'{base_url}{x}' for x in main_urls] # creates the link format correctly
    return(main_urls)
main_urls = get_links(main_url)
main_urls

['https://www.payscale.com/research/US/Job=ABAP_Developer/Salary',
 'https://www.payscale.com/research/US/Job=ASIC_Design_Engineer/Salary',
 'https://www.payscale.com/research/US/Job=ASIC_Engineer/Salary',
 'https://www.payscale.com/research/US/Job=ASP.NET_Developer/Salary',
 'https://www.payscale.com/research/US/Job=Actuarial_Associate/Salary',
 'https://www.payscale.com/research/US/Job=Actuarial_Consultant/Salary',
 'https://www.payscale.com/research/US/Job=Actuary/Salary',
 'https://www.payscale.com/research/US/Job=Administrator%2C_Lotus_Notes/Salary',
 'https://www.payscale.com/research/US/Job=Alliance_Manager%2C_Enterprise_Software/Salary',
 'https://www.payscale.com/research/US/Job=Analyst_Methods_%26_Procedures/Salary',
 'https://www.payscale.com/research/US/Job=Analytical_Strategist/Salary',
 'https://www.payscale.com/research/US/Job=Analytics_Manager/Salary',
 'https://www.payscale.com/research/US/Job=Android_Software_Developer/Salary',
 'https://www.payscale.com/research/US/J

In [6]:
print(f'I have {len(main_urls)} links to scrape')

I have 662 links to scrape


In [7]:
main_urls[0]

'https://www.payscale.com/research/US/Job=ABAP_Developer/Salary'

--- 
02. Saving the JSONs
--- 

In [8]:
# make function to extract JSON from HTML
def get_job(file):
    soup = read_cloud(file)
    data = json.loads(soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'}).text)
    return data

In [9]:
my_files = []  # create a list of file paths for later iteration

# Specify the directory path
directory_path = os.path.join(os.getcwd(), 'data_json')

# Create the directory if it doesn't exist
if not os.path.exists(directory_path):
    os.makedirs(directory_path)

for i, link in tqdm(enumerate(main_urls, 1)):
    page = read_cloud(link)
    data = get_job(link)  # get_job should return a dictionary, not convert it to a string

    # Specify the file path
    file_path = os.path.join(directory_path, f'payscale_IT_json_{i}.json')
    my_files.append(file_path)  # append to the list of file paths

    # Write the JSON data to the file using json.dump
    with open(file_path, 'w', encoding='utf-8') as my_file:
        json.dump(data, my_file, ensure_ascii=False, indent=4)

662it [04:40,  2.36it/s]


--- 
03. Defining function that would create a dict from JSON
--- 

In [10]:
def create_summary_df(path):
    with open(path, 'r',encoding='utf-8') as file:
        data = json.load(file)
        # Read the content of the file
        # data = file.read()

        dict_job = {}

        try:
            dict_job['job_title'] = data['props']['pageProps']['pageData']['dimensions']['job']
        except (KeyError, TypeError):
            dict_job['job_title'] = None

        try:
            dict_job['category'] = data['props']['pageProps']['pageData']['occupationalDetails']['occupationalCategory']
        except (KeyError, TypeError):
            dict_job['category'] = None

        try:
            dict_job['description'] = data['props']['pageProps']['pageData']['narratives']['description']
        except (KeyError, TypeError):
            dict_job['description'] = None

        try:
            dict_job['summary_hourly'] = data['props']['pageProps']['pageData']['narratives']['summaryHourly']
        except (KeyError, TypeError):
            dict_job['summary_hourly'] = None

        try:
            dict_job['summary_salary'] = data['props']['pageProps']['pageData']['narratives']['summarySalary']
        except (KeyError, TypeError):
            dict_job['summary_salary'] = None

        try:
            dict_job['country'] = data['props']['pageProps']['pageData']['country']
        except (KeyError, TypeError):
            dict_job['country'] = None

        try:
            dict_job['currency'] = data['props']['pageProps']['pageData']['currencyCode']
        except (KeyError, TypeError):
            dict_job['currency'] = None

        try:
            rating_data = data['props']['pageProps']['pageData']['ratings']['Job Satisfaction Overall']
            dict_job['rating'] = rating_data['score'] if rating_data is not None else None
        except (KeyError, TypeError):
            dict_job['rating'] = None

        try:
            rating_data = data['props']['pageProps']['pageData']['ratings']['Job Satisfaction Overall']
            dict_job['rating_count'] = rating_data['profileCount'] if rating_data is not None else None
        except (KeyError, TypeError):
            dict_job['rating_count'] = None

        try:
            gender_data = data['props']['pageProps']['pageData']['byDimension']['Gender Breakdown']
            dict_job['gender_responses'] = gender_data['profileCount'] if gender_data is not None else None
            gender_breakdown = gender_data['rows'] if gender_data is not None else []
            dict_job['male_count'] = gender_breakdown[0]['profileCount'] if len(gender_breakdown) > 0 else None
            dict_job['female_count'] = gender_breakdown[1]['profileCount'] if len(gender_breakdown) > 1 else None
        except (KeyError, TypeError):
            dict_job['gender_responses'] = None
            dict_job['male_count'] = None
            dict_job['female_count'] = None

    return dict_job

In [20]:
# Create a function for each dimension

def create_location_df(path):
    with open(path, 'r',encoding='utf-8') as file:
        data = json.load(file)
        # Read the content of the file
        # data = file.read()
        df_location = pd.DataFrame(data['props']['pageProps']['pageData']['byDimension']['Job by Location']['rows'])
        return df_location
    
def create_experience_df(path):
    with open(path, 'r',encoding='utf-8') as file:
        data = json.load(file)
        # Read the content of the file
        # data = file.read()
        df_experience = pd.DataFrame(data['props']['pageProps']['pageData']['byDimension']['Job by Experience']['rows'])
        return df_experience
    
def create_employer_df(path):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    try:
        employer_data = data['props']['pageProps']['pageData']['byDimension']['Job by Employer']['rows']
    except (KeyError, TypeError):
        employer_data = None

    df_employer = pd.DataFrame(employer_data)
    return df_employer
    
def create_skills_df(path):
    with open(path, 'r',encoding='utf-8') as file:
        data = json.load(file)
        # Read the content of the file
        # data = file.read()
        df_skill = pd.DataFrame(data['props']['pageProps']['pageData']['byDimension']['Job by Skill']['rows'])
        return df_skill

In [15]:
# create summary df and save it to csv

df_summary = [create_summary_df(x) for x in my_files]
df_summary = pd.DataFrame(df_summary)
df_summary.to_csv('data_summary.csv', index=False)
df_summary

,job_title,category,description,summary_hourly,summary_salary,country,currency,rating,rating_count,gender_responses,male_count,female_count
0,ABAP Developer,15-1030.00 - Computer Software Engineers,ABAP developers are in charge of designing and...,The average pay for an ABAP Developer is $82.1...,"The average pay for an ABAP Developer is $90,0...",US,USD,4.33,3.0,15.0,13.0,2.0
1,ASIC Design Engineer,15-1030.00 - Computer Software Engineers,An ASIC design engineer typically works within...,The average pay for an ASIC Design Engineer is...,The average pay for an ASIC Design Engineer is...,US,USD,3.91,11.0,31.0,26.0,4.0
2,ASIC Engineer,15-1030.00 - Unclassified Computer Software En...,ASIC (application-specific integrated circuit)...,None,"The average pay for an ASIC Engineer is $116,5...",US,USD,3.60,5.0,7.0,6.0,1.0
3,ASP.NET Developer,15-1030.00 - Computer Software Engineers,An ASP.NET developer must have experience and ...,The average pay for an ASP.NET Developer is $2...,The average pay for an ASP.NET Developer is $7...,US,USD,4.50,2.0,14.0,11.0,2.0
4,Actuarial Associate,15-2010.00 - Actuaries,Actuarial associates assist senior actuaries i...,None,The average pay for an Actuarial Associate is ...,US,USD,3.78,11.0,39.0,25.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...
657,Webmaster,15-1080.00 - Network Systems and Data Communic...,A webmaster’s general duties center around all...,The average pay for a Webmaster is $21.66 per ...,"The average pay for a Webmaster is $60,213 per...",US,USD,3.81,46.0,118.0,64.0,52.0
658,Website Manager,15-1080.00 - Network Systems and Data Communic...,Typical employers for website managers include...,The average pay for a Website Manager is $20.0...,"The average pay for a Website Manager is $68,0...",US,USD,3.30,25.0,80.0,45.0,34.0
659,Website Tester,15-1080.00 - Network Systems and Data Communic...,None,The average pay for a Website Tester is $25.00...,"The average pay for a Website Tester is $33,12...",US,USD,3.00,2.0,7.0,5.0,2.0
660,Wireless Communications Engineer,15-1080.00 - Network Systems and Data Communic...,A wireless communications engineer is a very s...,None,The average pay for a Wireless Communications ...,US,USD,3.00,1.0,7.0,5.0,2.0


In [16]:
# create location df and save it to csv

df_location = [create_location_df(x) for x in my_files]
df_location = pd.concat(df_location)
df_location.to_csv('data_location.csv', index=False)
df_location

,name,displayName,url,profileCount,range,isEstimated
0,"New York, New York",None,/research/US/Job=ABAP_Developer/Salary/a7aeab5...,10,{'50': 73758.95},None
1,"Atlanta, Georgia",None,/research/US/Job=ABAP_Developer/Salary/e4c182a...,8,{'50': 80000},None
2,"Minneapolis, Minnesota",None,/research/US/Job=ABAP_Developer/Salary/3a68bbf...,5,{'50': 104963.73},None
3,"Washington, District of Columbia",None,/research/US/Job=ABAP_Developer/Salary/fd511a1...,5,{'50': 67250},None
4,"Dallas, Texas",None,/research/US/Job=ABAP_Developer/Salary/a3ea7ab...,4,{'50': 105500},None
...,...,...,...,...,...,...
0,"Atlanta, Georgia",None,/research/US/Job=Wireless_Communications_Engin...,7,{'50': 82424.24},None
1,"Santa Clara, California",None,/research/US/Job=Wireless_Communications_Engin...,5,{'50': 120000},None
2,"San Diego, California",None,/research/US/Job=Wireless_Communications_Engin...,5,{'50': 97000},None
3,"New York, New York",None,/research/US/Job=Wireless_Communications_Engin...,5,{'50': 85000},None


In [17]:
# create experience df and save it to csv

df_experience = [create_experience_df(x) for x in my_files]
df_experience = pd.concat(df_experience)
df_experience.to_csv('data_experience.csv', index=False)
df_experience

,name,displayName,url,profileCount,range,isEstimated
0,Less than 1 year,None,/research/US/Job=ABAP_Developer/Salary/6b5bb51...,5,{'50': 57391.3},None
1,1-4 years,None,/research/US/Job=ABAP_Developer/Salary/6b5bb51...,10,{'50': 72696.54},None
2,5-9 years,None,/research/US/Job=ABAP_Developer/Salary/5357002...,17,{'50': 95595.1},None
3,10-19 years,None,/research/US/Job=ABAP_Developer/Salary/637c754...,14,{'50': 111411.79},None
4,20 years or more,None,/research/US/Job=ABAP_Developer/Salary/80a9e6f...,14,{'50': 101119.65},None
...,...,...,...,...,...,...
3,10-19 years,None,/research/US/Job=Wireless_Communications_Engin...,17,{'50': 101791.67},None
4,20 years or more,None,/research/US/Job=Wireless_Communications_Engin...,15,{'50': 92000},None
0,1-4 years,None,/research/US/Job=Wireless_Network_Optimization...,8,{'50': 65000},None
1,5-9 years,None,/research/US/Job=Wireless_Network_Optimization...,8,{'50': 72000},None


In [18]:
# create employer df and save it to csv

df_employer = [create_employer_df(x) for x in my_files]
df_employer = pd.concat(df_employer)
df_employer.to_csv('data_employer.csv', index=False)
df_employer

,name,displayName,url,profileCount,range,isEstimated
0,Intel Corporation,None,/research/US/Job=ASIC_Design_Engineer/Salary/c...,9,{'50': 111098.9},None
1,Lockheed Martin Corp,None,/research/US/Job=ASIC_Design_Engineer/Salary/f...,8,{'50': 78507.31},None
2,"Advanced Micro Devices, Inc.",None,/research/US/Job=ASIC_Design_Engineer/Salary/9...,6,{'50': 103000},None
3,Broadcom Corporation,None,/research/US/Job=ASIC_Design_Engineer/Salary/7...,5,{'50': 110000},None
0,"Juniper Networks, Inc.",None,/research/US/Job=ASIC_Engineer/Salary/178df232...,7,{'50': 123804.3},None
...,...,...,...,...,...,...
0,Georgia State University,None,/research/US/Job=Webmaster/Salary/074cf2bf/Geo...,8,{'50': 53599.1},None
1,West Virginia School of Osteopathic Medicine,None,/research/US/Job=Webmaster/Salary/963f589b/Wes...,7,{'50': 57500},None
2,Blockhouse,None,/research/US/Job=Webmaster/Hourly_Rate/5b3c9d2...,6,{'50': None},None
0,Boston Symphony Orchestra,None,/research/US/Job=Website_Manager/Salary/af507c...,6,{'50': 60000},None


In [19]:
# create skills df and save it to csv

df_skill = [create_skills_df(x) for x in my_files]
df_skill = pd.concat(df_skill)
df_skill.to_csv('data_skill.csv', index=False)
df_skill

,name,displayName,url,profileCount,range,isEstimated
0,SAP ABAP,None,/research/US/Job=ABAP_Developer/Salary/5d2db33...,31,{'50': 97551.68},None
1,SAP Enterprise Resource Planning (ERP),None,/research/US/Job=ABAP_Developer/Salary/4c1a29d...,13,{'50': 80000},None
2,Advanced Business Application Programming (ABA...,None,/research/US/Job=ABAP_Developer/Salary/f3707c7...,12,{'50': 80000},None
3,Object Oriented Programming (OOP),None,/research/US/Job=ABAP_Developer/Salary/7824f2d...,11,{'50': 85000},None
4,ABAP/4,None,/research/US/Job=ABAP_Developer/Salary/e789921...,7,{'50': 72500},None
...,...,...,...,...,...,...
7,Embedded / Real-Time / RTOS,None,/research/US/Job=Wireless_Communications_Engin...,5,{'50': 83473.05},None
0,Wireless,None,/research/US/Job=Wireless_Network_Optimization...,11,{'50': 70000},None
1,Cisco Networking,None,/research/US/Job=Wireless_Network_Optimization...,9,{'50': 65000},None
2,Wireless Data Communications,None,/research/US/Job=Wireless_Network_Optimization...,7,{'50': 110000},None
